In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop

### Pre-processing

In [2]:
nlp = spacy.load("fr_core_news_sm")
train_data = pd.read_csv("../archive/train.csv")
valid_data = pd.read_csv("../archive/valid.csv")
test_data  = pd.read_csv("../archive/test.csv")


In [3]:
train_data.head()

,Unnamed: 0,film-url,review,polarity
0,0,http://www.allocine.fr/film/fichefilm-135259/c...,Si vous cherchez du cinéma abrutissant à tous ...,0
1,1,http://www.allocine.fr/film/fichefilm-172430/c...,"Trash, re-trash et re-re-trash...! Une horreur...",0
2,2,http://www.allocine.fr/film/fichefilm-15105/cr...,"Et si, dans les 5 premières minutes du film, l...",0
3,3,http://www.allocine.fr/film/fichefilm-188629/c...,Mon dieu ! Quelle métaphore filée ! Je suis ab...,0
4,4,http://www.allocine.fr/film/fichefilm-23514/cr...,"Premier film de la saga Kozure Okami, ""Le Sabr...",1


**phase de tokenisation**

In [4]:
train_reviews = train_data.review.values
test_reviews  = test_data.review.values

On a changé le **step_words** pour utiliser le dictionnaire Francais

In [5]:
vectorizer = TfidfVectorizer(stop_words=list(fr_stop))
train_reviews_vectorised = vectorizer.fit_transform(train_reviews)
test_reviews_vectorised = vectorizer.transform(test_reviews)


/Users/amadouu/M2_ML/NLP/env_nlp/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['neuf', 'qu', 'quelqu'] not in stop_words.
  warnings.warn(


### Conception du modéle

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report

**phase d'entrainement du modéle**

In [7]:
y_train = train_data['polarity']
y_test  = test_data['polarity']

## defining the model
model = LogisticRegression()
model.fit(train_reviews_vectorised,y_train)
## Prediction on test data
y_pred = model.predict(test_reviews_vectorised)

In [8]:
print("Accuracy on test is:",accuracy_score(y_test,y_pred))
# classification report
report = classification_report(y_test, y_pred)
print(report)




Accuracy on test is: 0.92215
              precision    recall  f1-score   support

           0       0.93      0.92      0.92     10408
           1       0.91      0.93      0.92      9592

    accuracy                           0.92     20000
   macro avg       0.92      0.92      0.92     20000
weighted avg       0.92      0.92      0.92     20000



**Commentaires :**

Globalement, le modèle est performant avec une excellente précision et un bon rappel sur la plupart des classes. L'approche Tf-Idf a bien fonctionné ici.
- polarity 0: Le modèle identifie très bien les documents de cette classe, avec un rappel élevé (92%), ce qui signifie que peu de documents de cette classe sont manqués.
- polarity 1: Ce score est également très bon. 

Globalement, le modèle performe exceptionnellement bien sur cette catégorie, avec un F1-score de 0.92, ce qui est excellent.

### Création du pipeline

In [9]:
from sklearn.pipeline import Pipeline

estimators = [('pre_processing', TfidfVectorizer(stop_words=list(fr_stop))),('lrg', LogisticRegression())]
pipe = Pipeline(estimators)
pipe

Pipeline(steps=[('pre_processing',
                 TfidfVectorizer(stop_words=['certes', 'la', 'mêmes',
                                             'allaient', 'eu', 'chacun',
                                             'autrui', 'différents', 'ouias',
                                             'première', 'ces', 'possible',
                                             'tes', 'lui', 'revoila',
                                             'maintenant', 'outre', 'parlent',
                                             'quelconque', 'precisement', 'bas',
                                             'moi-même', 'ouvert', 'suivants',
                                             'etc', 'ainsi', 'ton', 'ha',
                                             'sera', 'et', ...])),
                ('lrg', LogisticRegression())])

In [10]:
pipe.fit(train_reviews,y_train)
y_pred_v2 = pipe.predict(test_reviews)
report = classification_report(y_test, y_pred_v2)
print(report)

/Users/amadouu/M2_ML/NLP/env_nlp/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['neuf', 'qu', 'quelqu'] not in stop_words.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.93      0.92      0.92     10408
           1       0.91      0.93      0.92      9592

    accuracy                           0.92     20000
   macro avg       0.92      0.92      0.92     20000
weighted avg       0.92      0.92      0.92     20000



In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

On a les mêmes scores que dans la section précédente

### Expérimentation de différents modéles et hyperparamétres

Regression Logistique : c'est un modéle de classification qui performe plus pour les problémes de classfication binaire.
On a deux paramêtres principaux : **C** et **penalty**

In [12]:
MAX_ITER = 500
scores = []
parameters = {'C': [0.01, 0.1, 1, 10, 100], 'penalty': ['l1','l2']}
for c in parameters.get('C'):
    for pen in parameters.get('penalty'):
        lg = LogisticRegression(C=c, penalty=pen, max_iter=MAX_ITER, solver='liblinear', random_state=42)
        lg.fit(train_reviews_vectorised,y_train)
        ## Prediction on test data
        y_pred = lg.predict(test_reviews_vectorised)
        report = classification_report(y_test, y_pred, output_dict=True)
        # Stockage des scores
        scores.append((c, pen, report['0']['f1-score'], report['1']['f1-score']))




In [13]:
scores

[(0.01, 'l1', 0.7517737951332317, 0.7748057025699709),
 (0.01, 'l2', 0.8853751046746465, 0.8818721762526016),
 (0.1, 'l1', 0.8864801523363117, 0.880885291254675),
 (0.1, 'l2', 0.9102501703494598, 0.9052122956718412),
 (1, 'l1', 0.9202328966521106, 0.9152140278494069),
 (1, 'l2', 0.9246153099364108, 0.9199443270271663),
 (10, 'l1', 0.914178204447875, 0.907685122723263),
 (10, 'l2', 0.9227347451890533, 0.917279221451496),
 (100, 'l1', 0.8815257175881334, 0.8721147847785402),
 (100, 'l2', 0.9059556586002029, 0.8991034875887444)]

MLP: c'est un modèle de réseau de neurones adapté à des jeux de données plus complexes où les classes ne sont pas linéairement séparables.
hidden_layer_sizes => nombre de couches cachées
alpha ~ C

In [ ]:
mlp_parameters = {
    'hidden_layer_sizes': [(5,), (10,10), (15, 15)],
    'alpha': [0.0001, 0.001, 0.01]
}
mlp_scores = []
for hl in mlp_parameters['hidden_layer_sizes']:
    for a in mlp_parameters['alpha']:
        mlp = MLPClassifier(hidden_layer_sizes=hl, alpha=a, early_stopping=True, random_state=42)
        mlp.fit(train_reviews_vectorised, y_train)        
        y_pred = mlp.predict(test_reviews_vectorised)
        report = classification_report(y_test, y_pred, output_dict=True)        
        mlp_scores.append((hl, a, report['0']['f1-score'], report['1']['f1-score']))
mlp_scores

/Users/amadouu/M2_ML/NLP/env_nlp/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:697: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


SVM : c'est une méthode de classification très efficace pour les jeux de données où les classes sont bien séparables.
kernel => pour le noyau
C => pour la régulation

In [ ]:
svm_parameters = {
    'C': [0.01, 0.1, 1, 10],
    'kernel': ['linear', 'rbf']
}
svm_scores = []
for c in svm_parameters['C']:
    for k in svm_parameters['kernel']:
        svm = SVC(C=c, kernel=k, random_state=42)
        svm.fit(train_reviews_vectorised, y_train)        
        y_pred = svm.predict(test_reviews_vectorised)
        report = classification_report(y_test, y_pred, output_dict=True)        
        svm_scores.append((c, k, report['0']['f1-score'], report['1']['f1-score']))
svm_scores